# Creating the texts

Ziel: 
* Texte importieren, 
* in chunks unterteilen (Kapitel), 
* Texte bereinigen
* jedes Kapitel mit cltk bearbeiten
* als einzelfile speichern.


## Importing txt-files

The text of the SBLGNT-txt-files looks like this:
"ΚΑΤΑ ΜΑΡΚΟΝ
Mark 1:1	Ἀρχὴ τοῦ εὐαγγελίου Ἰησοῦ ⸀χριστοῦ. 
Mark 1:2	⸀Καθὼς γέγραπται ἐν ⸂τῷ Ἠσαΐᾳ τῷ προφήτῃ⸃· ⸀Ἰδοὺ ἀποστέλλω τὸν ἄγγελόν μου πρὸ προσώπου σου, ὃς κατασκευάσει τὴν ὁδόν ⸀σου· 
"

With the following code, the first line is dropped and the text is split into individual chapters. It returns a dictionary with the key: bookname chapter (e.g. "Mark 1"), and the value the text of the chapter.

In [1]:
import re
import string

def splitSBLGNT(collection, bookname):
    '''
    takes an sblgnt.txt file, splits into chapters
    returns a dictionary (keys: "Mark 1" etc.)
    '''
    
    with open('data_raw/' + collection + '/' + bookname + '.txt') as f:
        book = f.read()
    book = book[book.find(bookname):] # drop the book name from file ("Kata Markon etc.")
    
    #remove punctation and textcritical markers
    punctuation = '!"#$%&\()*+,-./;<=>?@[\\]^_`{|}~·⸀⸀⸂⸃'
    text = book.translate(str.maketrans("", "", punctuation))
        
    
    chapters = {}  # Dictionary to store chapters

    # Split the text into lines
    lines = text.strip().split('\n')

    current_chapter = ""
    current_text = ""

    for line in lines:
        parts = re.split(r'(:\d+)\s+', line, maxsplit=1)  # Split each line into chapter, verse and text
        chapter = parts[0]  # Extract chapter information
        verse_text = parts[2] if len(parts) > 1 else ""  # Text of the verse (if available)

        if chapter != current_chapter:  # If a new chapter is encountered
            if current_chapter:
                chapters[current_chapter] = current_text.strip()  # Store the previous chapter's text
            current_chapter = chapter
            current_text = verse_text
        else:
            current_text += " " + verse_text  # Concatenate verse text to current chapter

    # Store the last chapter
    chapters[current_chapter] = current_text.strip()

    
    return(chapters)    


In [2]:
Mt = splitSBLGNT("SBLGNT", "Matt")
Mk = splitSBLGNT("SBLGNT", "Mark")
Lk = splitSBLGNT("SBLGNT", "Luke")
Joh = splitSBLGNT("SBLGNT", "John")
Act = splitSBLGNT("SBLGNT", "Acts")

## CLTK annotation

In [4]:
import cltk
from cltk import NLP
#from cltk.dependency.tree import DependencyTree
from cltk.languages.pipelines import GreekPipeline

import pandas as pd

cltk_nlp = NLP(language="grc")

‎𐤀 CLTK version '1.3.0'. When using the CLTK in research, please cite: https://aclanthology.org/2021.acl-demo.3/

Pipeline for language 'Ancient Greek' (ISO: 'grc'): `GreekNormalizeProcess`, `GreekSpacyProcess`, `GreekEmbeddingsProcess`, `StopsProcess`.

⸖ ``GreekSpacyProcess`` using OdyCy model by Center for Humanities Computing Aarhus from https://huggingface.co/chcaa . Please cite: https://aclanthology.org/2023.latechclfl-1.14
⸖ ``LatinEmbeddingsProcess`` using word2vec model by University of Oslo from http://vectors.nlpl.eu/ . Please cite: https://aclanthology.org/W17-0237/

⸎ To suppress these messages, instantiate ``NLP()`` with ``suppress_banner=True``.


In [5]:
def CLTKannotateDic(dictionary):
    newdictionary = {}
    for i in dictionary:
        newdictionary[i] = cltk_nlp.analyze(text=dictionary[i])
    return(newdictionary)

Save the CLTK doc-objects as Python pickles:

In [6]:
def Word2Pickle(wordObject, bookname):
    import pickle
    data = wordObject
    with open("data_annotated/SBLGNT/" + bookname + ".pkl", "wb") as pickle_file:
        pickle.dump(data, pickle_file)

Combine the annotate function with the word2Pickle function:

In [7]:
def annotate2Pickle(dictionary, bookname):
    bookname = bookname + "_Cltk"
    cltk = CLTKannotateDic(dictionary)
    Word2Pickle(cltk, bookname)
    

Function calls:

In [9]:
annotate2Pickle(Mk, "Mk")

/opt/anaconda3/envs/CLTK_NT/lib/python3.9/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [34]:
annotate2Pickle(Mt, "Mt")

In [35]:
annotate2Pickle(Lk, "Lk")

In [36]:
annotate2Pickle(Joh, "Joh")

In [37]:
annotate2Pickle(Act, "Acts")

To open and load the pickle file:

In [10]:
import pickle
mkpkl = pickle.load( open( "data_annotated/SBLGNT/Mk_Cltk.pkl", "rb" ) )

In [32]:
len(mkpkl["Mark 16"].lemmata)

360

In [33]:
len(mkpkl["Mark 16"].tokens)


360